In [13]:
from resources.filterLang import FilterLanguage as FL
from resources.tokTT import CommentTokenizer as CT
import fasttext
import numpy as np
from resources.basicIO import InputOutput as IO
import copy
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

In [2]:
def oracleHelp(classdata):

    sums = sum(classdata)
    res = any(((ele/sums) >= 0.48 and (ele/sums) <= 0.52) for ele in classdata)
    return res

In [3]:
def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

In [4]:
def sim(x,y,sim_type='cosine_sim'):

    if(sim_type == 'cosine_sim'):
        return cos_sim(x,y)

### load model

In [5]:
model_N_2 = Word2Vec.load("models/word2vec.model")

### load comments, seedset, labels of both

In [6]:
# loading 60 comments
expansion_text = IO.load_csv_col('datasets/random_sample.csv', 'comment')
expansion_text = expansion_text[0:500]
expansion_text_labels = IO.load_csv_col('datasets/random_sample.csv', 'label')
expansion_text_labels = list(map(int, expansion_text_labels[0:500]))
seed_set_text = IO.load_text('datasets/seed_set.txt')
seed_set_labels = IO.load_text('datasets/seed_set_labels.txt')
seed_set_labels = list(map(int, seed_set_labels))
seed_set_TK = CT.cleaned('datasets/seed_set.txt')
expansion_TK = [CT.tokenize(x) for x in expansion_text]

In [7]:
seed_text = copy.deepcopy(seed_set_text)
seed_labels = copy.deepcopy(seed_set_labels)

In [10]:
type(model_N_2)

gensim.models.word2vec.Word2Vec

In [14]:
def filter_keys_list(model: Word2Vec, A: list):

    lst = []

    for key in A:
        if model.wv.has_index_for(key):
            lst.append(key)
    return lst

In [9]:
for i in range(len(expansion_TK)):
    maxscore = 0.0
    lbl = 0
    for j in range(len(seed_set_TK)):
        A = filter_keys_list(model_N_2, expansion_TK[i].split())
        B = filter_keys_list(model_N_2, seed_set_TK[j].split())
        score = model_N_2.wv.n_similarity(A,B)
        if(score > maxscore):
            maxscore = score
            lbl = seed_labels[j]

    # push
    seed_labels.append(seed_set_labels[lbl])
    seed_text.append(expansion_text[i])
    seed_set_TK.append(expansion_TK[i])

KeyError: "Key 'is' not present"

In [ ]:
IO.save_text('datasets_post/word2vec_max_inbuilt.txt', seed_text)
IO.save_text('datasets_post/word2vec_max_inbuilt_labels.txt',map(str, seed_labels))